<a href="https://colab.research.google.com/github/yagnyaPatel/Chandras-Angels/blob/master/Campaign_Clicks_Pred_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab as pl
from pandas.plotting import scatter_matrix
from matplotlib import cm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
from datetime import datetime
import pytz
from dateutil import tz
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
#
# sample_####_join.csv it's the data of the clicks and impressions of the campaign 
#
# In this part merge function it's to join the impressions and clicks table
#
sample_clicks = pd.read_csv("ojpro_q4_fy18_clicks.csv", low_memory = False);
sample_impressions = pd.read_csv("ojpro_q4_fy18_impressions.csv", low_memory = False);
sample_clicks = sample_clicks.drop(['key','browser_id','region_id',
                                                'placement', 'creative_type'], axis=1);
sample = pd.merge(sample_impressions, sample_clicks, how='left').fillna(0);
sample = sample.fillna(0);
sample = sample[sample.impressions!= 1];
sample['CTR'] = (sample['clicks']/sample['impressions'])*100;
#sample.head()
print("Clicks=",sum(sample['clicks']))
print()
print("Impressions=",sum(sample['impressions']))
print()
print("CTR=",(sum(sample['clicks'])/sum(sample['impressions']))*100)

FileNotFoundError: ignored

In [0]:
#
# This function is to obtain the timezone of every state or region of US and CA
#
def f_timezone(row):
    if row['state_region'] in ['AK']:
        val = 'US/Alaska'
    elif row['state_region'] in ['AL', 'AR', 'IA', 'IL', 'KS', 'LA', 'MN', 'MO', 'MS', 'ND', 'NE', 'OK','SD', 'TN','TX','WI']:
        val = 'US/Central'
    elif row['state_region'] in ['CT','DC','DE','FL','GA','IN','KY','MA','MD','ME','MI','NC',
                                 'NH','NJ','NY','OH','ON','PA','QC','RI','SC','VA','VT','WV']:
        val = 'US/Eastern'
    elif row['state_region'] in ['AB','AZ','CO','ID','MT','NM','UT','WY']:
        val = 'US/Mountain'
    elif row['state_region'] in ['BC','CA','NV','OR','WA']:
        val = 'US/Pacific'
    elif row['state_region'] in ['HI']:
        val = 'US/Hawaii'
    else:
        val = 'UTC'
    return val
#
# This function is to obtain the real date and time of each time zone
#
def f_convert(row):
    if row['Timezone'] in ['US/Alaska', 'US/Central', 'US/Eastern', 'US/Mountain','US/Pacific','US/Hawaii']:
        val = row['date'].tz_localize('UTC').tz_convert(row['Timezone']).tz_localize(None)
    else:
        val = row['date'] 
    return val
#
# The next function creates a new class depending on the real time and the part of the day divided into 6 parts of 4 hrs
#
def f_day(row):
    if row['hour'] in [23, 0, 1, 2] :
        val = 'Middle_Night'
    elif row['hour'] in [3, 4, 5, 6] :
        val = 'Early_Morning'
    elif row['hour'] in [7, 8, 9, 10] :
        val = 'Morning'
    elif row['hour'] in [11, 12, 13, 14]:
        val = 'Midday'
    elif row['hour'] in [15, 16, 17, 18] :
        val = 'Evening'
    elif row['hour'] in [19, 20, 21, 22] :
        val = 'Night'
    return val
#
# The next functions creates a new class depending on the CTR
#
##data = data.dropna(); 
#data = data.fillna('NAN');
def f_CTR(row):
    if row['CTR'] == 0:
        val = 0
    elif 0 < row['CTR'] <= 0.1:
        val = 1
    elif 0.1 < row['CTR'] <= 0.1:
        val = 2
    elif 0.1 < row['CTR'] <= 25:
        val = 3
    elif 25 < row['CTR'] <= 50:
        val = 4
    elif 50 < row['CTR'] <= 75:
        val = 5
    elif 75 < row['CTR'] <= 100:
        val = 6
    else:
        val = 0
    return val

def f_CTR2(row):
    if row['CTR'] == 0:
        val = 0
    elif 0 < row['CTR'] <= 0.1:
        val = 1
    elif 0.1 < row['CTR'] <= 1:
        val = 2
    elif 1 < row['CTR'] <= 100:
        val = 3
    else:
        val = 0
    return val

def f_CTR3(row):
    if row['CTR'] == 0:
        val = 0
    else:
        val = 1
    return val


#
# The next functions creates a new class depending on the operating_system to get the device info
#
def device(row):
    if any(word in row['operating_system'] for word in ['Microsoft', 'Unix', 'Linux', 'Macintosh']):
        val = 'PC/Mac'
    elif any(word in row['operating_system'] for word in ['Android', 'Apple iOS', 'Windows Phone',
                                                          'Nokia','BlackBerry', 'Palm', 'DoCoMo', 'Samsung']):
        val = 'Mobile'
    else:
        val = 'Others'
    return val

#
# Function to separe the placement string
#
def f_audience(row):
    if len(row['placement'].split('_')) == 7:
        val = row.str.split('_').str[2]
    elif len(row['placement'].split('_')) == 8:
        val = row.str.split('_').str[2]
    else:
        val = 0
    return val

def f_dates(row):
    if len(row['placement'].split('_')) == 7:
        val = row.str.split('_').str[3]
    elif len(row['placement'].split('_')) == 8:
        val = row.str.split('_').str[4]
    else:
        val = 0
    return val

def f_ad_size(row):
    if len(row['placement'].split('_')) == 7:
        val = row.str.split('_').str[4]
    else:
        val = row.str.split('_').str[5]
    return val

In [0]:
sample['Device'] = sample.apply(device, axis=1);
sample['CTR_Class'] = sample.apply(f_CTR, axis=1);
sample['CTR_Class2'] = sample.apply(f_CTR2, axis=1);
sample['Timezone'] = sample.apply(f_timezone, axis=1);
sample['date'] =  pd.to_datetime(sample['date'], format="%Y/%m/%d-%H")
sample['local_date'] = sample.apply(f_convert, axis=1)
sample['Day_id'] = pd.to_datetime(sample['local_date'], format='%Y-%m-%dT%H:%M:%S').dt.strftime('%w');
sample['Day'] = pd.to_datetime(sample['local_date'], format='%Y-%m-%dT%H:%M:%S').dt.strftime('%A');
sample['hour'] = pd.to_datetime(sample['local_date'], format='%Y-%m-%dT%H:%M:%S').dt.hour;
sample['Day_Part'] = sample.apply(f_day, axis=1);

if sample['placement'].astype(str).str.split('_').map(len) == 8:
        sample['placement_ad_size'] = sample['placement'].str.split('_').str[5]
        sample['placement_audience'] = sample['placement'].str.split('_').str[2]
        sample['placement_date'] = sample['placement'].str.split('_').str[4]    
else:
        sample['placement_ad_size'] = sample['placement'].str.split('_').str[4]
        sample['placement_audience'] = sample['placement'].str.split('_').str[2]
        sample['placement_date'] = sample['placement'].str.split('_').str[3]
        
#sample['placement_ad_size'].replace('080118-091518', '1x1', inplace=True);    
sample.head() 

In [0]:
sample.groupby('placement_ad_size').placement_ad_size.count()

In [0]:
len(sample['placement'].iloc[10].split('_')) 

In [0]:
sample['adsize'] = sample.apply(f_ad_size, axis=1);

In [0]:
sample.to_csv('ojpro_full_sample_q4_fy18.csv');

In [0]:
print("Clicks=",sum(sample['clicks']))
print()
print("Impressions=",sum(sample['impressions']))
print()
print("CTR=",(sum(sample['clicks'])/sum(sample['impressions']))*100)

In [0]:
##########################################################################################################################   
##########################################################################################################################
## If the full sample is already in the folder, Start here ##
##########################################################################################################################    
##########################################################################################################################    

In [0]:
sample = pd.read_csv("ojpro_full_sample_q4_fy18.csv", low_memory = False);

In [0]:
print("Clicks=",sum(sample['clicks']))
print()
print("Impressions=",sum(sample['impressions']))
print()
print("CTR=",(sum(sample['clicks'])/sum(sample['impressions']))*100)

In [0]:
def f_CR(row):
    if row['CTR'] == 0  :
        val = 0
    else:
        val = 1
    return val
sample['CR'] = sample.apply(f_CR, axis=1);

In [0]:
sample3 = sample.drop(sample[(sample['clicks'] == 0) & (sample['impressions'] <= 30)].index)
sample3 = sample3.drop(sample3[(sample3['CTR_Class2'] == 1)].index)
#sample3 = sample;
sample3[['ad_size_id']] = sample3[['placement_ad_size']].apply(lambda x: pd.factorize(x)[0] + 1)
sample3[['Timezone_id']] = sample3[['Timezone']].apply(lambda x: pd.factorize(x)[0]+ 1 )
sample3[['day_part_id']] = sample3[['Day_Part']].apply(lambda x: pd.factorize(x)[0] + 1)
sample3[['Device_id']] = sample3[['Device']].apply(lambda x: pd.factorize(x)[0] + 1)
features = ['ad_size_id','Timezone_id','day_part_id', 'region_id', 'hour','Day_id', 'browser_id', 'Device_id']
target = 'CR';

In [0]:
print("Percentage of CTR Class in full sample:")
print()
print(sample['CR'].value_counts(normalize=True) * 100)
print()
print("Clicks=",sum(sample['clicks']))
print()
print("Impressions=",sum(sample['impressions']))
print()
print("CTR=",(sum(sample['clicks'])/sum(sample['impressions']))*100)
print()
print("Percentage of CTR Class better distributed:")
print()
print(sample3['CR'].value_counts(normalize=True) * 100)
print()
print("Clicks=",sum(sample3['clicks']))
print()
print("Impressions=",sum(sample3['impressions']))
print()
print("CTR=",(sum(sample3['clicks'])/sum(sample3['impressions']))*100)

In [0]:
X = sample3[features]
y = sample3['CTR_Class2']

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.25, random_state = 10)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

rfs= RandomForestClassifier(n_estimators = 100, max_depth = 20, random_state=42);
rfs.fit(X_train, y_train);

In [0]:
predictions = rfs.predict(X_test)
#print("Score:", rfs.score(y_test, predictions))
metrics.accuracy_score(y_test, predictions)

In [0]:
#Precision in the training sample
print("Training Precision: {0: .4f}".format(
        rfs.score(X_train, y_train)))

In [0]:
#Precision in the evaluation sample 
print("Evaluation precision: {0: .4f}".format(
        rfs.score(X_test, y_test)))

In [0]:
sample3.head()

In [0]:
sample3.groupby('Device').Device.count()

In [0]:
def one_hot_obj_features(df, features):
    new_df = pd.get_dummies(df, columns=features, sparse=True)
    return new_df

In [0]:
features = ['placement_ad_size','Timezone','Day_Part', 'state_region', 'hour','Day', 'browser_platform', 'Device']
target = 'CR';
train_sample = sample3[features + [target]].sample(frac=0.25, random_state=21)
train_sample = one_hot_obj_features(train_sample, ['placement_ad_size','Timezone','Day_Part', 
                                                   'state_region', 'hour','Day', 'browser_platform', 'Device'])

model_features = np.array(train_sample.columns[train_sample.columns != target].tolist())
X = train_sample[model_features].values;
y = train_sample[target].values;

X_train, X_test, y_train, y_test = train_test_split(
    train_sample[model_features].values, 
    train_sample[target].values,
    test_size=0.25,
    random_state=21
)

rfs= RandomForestClassifier(n_estimators = 100, max_depth = 50, random_state=42);
rfs.fit(X_train, y_train);

In [0]:
predictions = rfs.predict(X_test)
#print("Score:", rfs.score(y_test, predictions))
metrics.accuracy_score(y_test, predictions)

In [0]:
rfs.fit(X_train, y_train);
rfs_cv_score = cross_val_score(rfs, X, y, cv=5, scoring='accuracy')

In [0]:
print('Accuracy of Random Forest classifier on training set: {:.2f}'
     .format(rfs.score(X_train, y_train)))
print('Accuracy of Random Forest classifier on test set: {:.2f}'
     .format(rfs.score(X_test, y_test)))
print()
print()
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, predictions))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, predictions))
print('\n')
print("=== All Accuracy Scores ===")
print(rfs_cv_score)
print('\n')
print("=== Mean }Accuracy Score ===")
print("Mean Accuracy Score - Random Forest: ", rfs_cv_score.mean())

In [0]:
cross_val_score(rfs, X, y, cv=4).mean()

In [0]:
scores = cross_val_score(rfs, X, y, cv=6)
scores

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

#feature_names = ['ad_size_id','Timezone_id','day_part_id', 'region_id', 'hour','Day_id', 'browser_id', 'Device_id']
#X = sample3[feature_names]
#y = sample3['CTR_Class2']

kfold = model_selection.KFold(n_splits=10, random_state=21)
scoring = 'accuracy'
####################################################################
#######  Create Training and Test Sets and Apply Scaling ###########
####################################################################
#X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.25, random_state = 10)
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test) 
####################################################################
#######################  Logistic Regression #######################
####################################################################
logreg = LogisticRegression(solver='lbfgs',multi_class='auto')
logreg.fit(X_train, y_train)
print()
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
      .format(logreg.score(X_train, y_train)))
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
      .format(logreg.score(X_test, y_test)))
results = model_selection.cross_val_score(logreg, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
print()
print("Logistic Regression")
pred = logreg.predict(X_test)
print()
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred))
####################################################################
#######################    Decision Tree     #######################
####################################################################
dtc = DecisionTreeClassifier().fit(X_train, y_train)
print()
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(dtc.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(dtc.score(X_test, y_test)))
results = model_selection.cross_val_score(dtc, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
####################################################################
######################  K-Nearest Neighbors  #######################
####################################################################
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print()
print('Accuracy of K-Nearest Neighbors classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-Nearest Neighbors classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))
results = model_selection.cross_val_score(knn, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
print()
print("K-Nearest Neighbors")
pred = knn.predict(X_test)
print()
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred))
####################################################################
#############  Linear Discriminant Analysis  #######################
####################################################################
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print()
print('Accuracy of Linear Discriminant Analysis classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of Linear Discriminant Analysis classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))
results = model_selection.cross_val_score(lda, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
####################################################################
#####################  Gaussian Naive Bayes  #######################
####################################################################
gnb = GaussianNB()
gnb.fit(X_train, y_train)
print()
print('Accuracy of Gaussian NB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, y_train)))
print('Accuracy of Gaussian NB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, y_test)))
results = model_selection.cross_val_score(gnb, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
####################################################################
###################  Support Vector Machine  #######################
####################################################################
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
print()
print('Accuracy of Support Vector Machine classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of Support Vector Machine classifier on test set: {:.2f}'
     .format(svm.score(X_test, y_test)))
results = model_selection.cross_val_score(svm, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
print()
print("Support Vector Machine")
pred = svm.predict(X_test)
print()
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred))
####################################################################
######################     Random Forest    #######################
####################################################################
rf = RandomForestClassifier(n_estimators = 500)
rf.fit(X_train, y_train)
print()
print('Accuracy of Random Forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))
print('Accuracy of Random Forest classifier on test set: {:.2f}'
     .format(rf.score(X_test, y_test)))
results = model_selection.cross_val_score(rf, X, y, cv=kfold, scoring=scoring)
print("Accuracy (Cross Validation Classification Accuracy):", results.mean())
print()
####################################################################
######################                       #######################
####################################################################
print()
print("Decision Tree")
pred = dtc.predict(X_test)
print()
print(confusion_matrix(y_test, pred))
print()
print(classification_report(y_test, pred))
print()
print()
print("Random Forest")
pred2 = rf.predict(X_test)
print()
print(confusion_matrix(y_test, pred2))
print()
print(classification_report(y_test, pred2))
print()


In [0]:
pd.crosstab(y_test,pd.Series(pred2),rownames=['ACTUAL'],colnames=['PRED'])